# GPT-2 From Scratch

This is my implementation of GPT-2 (124M) based on Andrej Karpthy's tutorial on YouTube, along with my notes on the same. 

Initially, we're going to build exact replica of GPT-2. But then we will train the model to improve on this perhaps. Here, we reproduce the GPT-2 as it is but we are going to create maybe different class names, etc. The naming of the parameters in all the module classes follows the `transformers` library's state dictionary. 

In [ ]:
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

class CausalSelfAttention(nn.Module):
    def __init__(self, config:GPTConfig):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)

        self.n_head = config.n_head
        self.n_embd = config.n_embd

        # 4D shape now
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size)) 

    def forward(self, x):
        B, T, C = x.size() # Batch Size, Sequence Length, Embedding Dim

        qkv = self.c_attn(x)

        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        y = self.c_proj(y)

        return y


class MLP(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()

        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):

    def __init__(self, config:GPTConfig):
        super().__init__()
        self.config = config
        
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # text embeddings
            wpe = nn.Embedding(config.block_size, config.n_embd), # positional embeddings
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]), 
            ln_final = nn.LayerNorm(config.n_embd)
        )) 

        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, idx):
        B, T, C = idx.size()

        assert T <= self.config.block_size

        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = pos_emb + tok_emb

        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_final(x)

        logits = self.lm_head(x)

        return logits


### Few Notes:

1. GeLU's approximate version was used in GPT-2. The reason for this was that the exact version was quite slow in Tensorflow, where GPT-2 was trained. But there is no real reason now to not use exact version of GELU. So, when we try to improve this model, we will use GELU in its exact form.

2. Organization of `LayerNorm` layers was changed in GPT-2 from the Attention is All You Need paper. (see Shakespeare in me). But in the original paper, `LayerNorm` was kept inside the residual block which is not desirable because you want your residual stream to be just additions to make the gradients flow. 

3. Attention is aggregation operation where information goes across tokens. But in the MLP layer, operation is performed on each token individually. 

4. Causal Self Attention is a term given to attention where you only look at the previous tokens. As opposed to Masked Self Attention where you are allowed to look at both sides but some tokens are masked. 

5. In the Shakespeare code, we had multiple modules- one for Head, one for concatenating the multiple heads, etc. Here, we are just having all of it in one module by doing some smart `.view()` operations. 

What we have above is a complete GPT-2 implementation. If we can somehow get the weights from `transformers` and add them to our model, we should get a similar performance as GPT-2.